In [2]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


#### Sistema de recomendación item-item:

- **def recomendacion_juego( item_id ):**

> Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.


In [3]:

# Ruta al archivo Parquet
file_path = 'datasets/processed_data/ml_genres.parquet'

# Leer el archivo Parquet
table = pq.read_table(file_path)

# Convertir la tabla a un DataFrame de pandas
df_genres = table.to_pandas()


# Ruta al archivo Parquet
file_path = 'datasets/processed_data/steam_games.parquet'

# Leer el archivo Parquet
table = pq.read_table(file_path)

# Convertir la tabla a un DataFrame de pandas
df_steam_Games = table.to_pandas()

In [4]:
df_genres

,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,Free to Play,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
item_id,,,,,,,,,,,,,,,,,,,,,
10,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100400,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10090,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100980,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99700,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9980,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99900,0,1,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [5]:
def recomendacion_juego(item_id):
    # Obtener la fila correspondiente al item dado
    item_row = df_genres.loc[item_id].values.reshape(1, -1)
    
    # Calcular la similitud coseno entre el item dado y todos los demás items
    similarities = cosine_similarity(item_row, df_genres.values)
    
    # Crear un DataFrame de similitudes con los item_id como índice
    sim_df = pd.DataFrame(similarities, columns=df_genres.index, index=['similarity']).T
    
    # Ordenar los items por similitud en orden descendente y eliminar el item dado
    sim_df = sim_df.drop(item_id).sort_values(by='similarity', ascending=False)
    
    # Obtener los 5 items más similares
    top_5_similar_items = sim_df.head(5)
####
    # Realizar un merge con df_steam_Games para obtener los títulos de los juegos
    top_5_similar_items_with_title = pd.merge(top_5_similar_items, df_steam_Games, left_index=True, right_on='item_id')
    
    # Filtrar los títulos de los juegos recomendados
    recommended_titles = top_5_similar_items_with_title['title'].tolist()
    
    return recommended_titles
    

In [6]:
recomendacion_juego("17140")

["King's Bounty: Warriors of the North - Ice and Fire",
 'Endless Legend™ - Echoes of Auriga',
 "Baldur's Gate: Siege of Dragonspear",
 'SpellForce 2 - Demons of the Past - Soundtrack']

#### Sistema de recomendación user-item:

- **def recomendacion_usuario( user_id):**

> Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [7]:
# Especifica la ruta del archivo Parquet
ruta_archivo_parquet = 'datasets/processed_data/ml_user_items.parquet'

# Lee el archivo Parquet en una tabla de PyArrow
tabla_parquet = pq.read_table(ruta_archivo_parquet)

# Convierte la tabla a un DataFrame de pandas si es necesario
ml_user_items = tabla_parquet.to_pandas()


In [8]:
ml_user_items

,user_id,item_id,playtime_forever
0,syates,1002,121
1,76561197973196808,1002,29
2,76561198007521657,1002,49
3,Blarghargh,1002,64
4,76561197969407671,1002,9
...,...,...,...
675015,Rhuri_AGA,9980,121
675016,romakin,9980,10
675017,TandyTandon,9980,21
675018,2cq,9980,25


In [9]:
from sklearn.preprocessing import MinMaxScaler

# Normalizar playtime_forever en un rango de 0 a 1
scaler = MinMaxScaler()
# Crear una nueva columna para dicha normalización
ml_user_items['normalized_playtime'] = scaler.fit_transform(ml_user_items[['playtime_forever']])

# Eliminar las columnas playtime_forever
ml_user_items = ml_user_items.drop(['playtime_forever'], axis=1)

# elimnar duplicados
ml_user_items = ml_user_items.drop_duplicates(subset=['item_id', 'user_id'])



In [13]:
ml_user_items

,user_id,item_id,normalized_playtime
0,syates,1002,0.094044
1,76561197973196808,1002,0.021944
2,76561198007521657,1002,0.037618
3,Blarghargh,1002,0.049373
4,76561197969407671,1002,0.006270
...,...,...,...
675015,Rhuri_AGA,9980,0.094044
675016,romakin,9980,0.007053
675017,TandyTandon,9980,0.015674
675018,2cq,9980,0.018809
